# 65536


In [2]:
import pygame
import random
import copy
import time

# Define some constants for the game
GRID_SIZE = 4
TILE_SIZE = 100
WINDOW_SIZE = TILE_SIZE * GRID_SIZE
BG_COLOR = (187, 173, 160)
TILE_COLORS = {
    0: (205, 193, 180),
    2: (238, 228, 218),
    4: (237, 224, 200),
    8: (242, 177, 121),
    16: (245, 149, 99),
    32: (246, 124, 95),
    64: (246, 94, 59),
    128: (237, 207, 114),
    256: (237, 204, 97),
    512: (237, 200, 80),
    1024: (237, 197, 63),
    2048: (237, 194, 46)
}

# utility functions

def get_direction_text(direction):
    return "up" if direction == 0 else "right" if direction == 1 else "down" if direction == 2 else "left" if direction == 3 else direction

pygame 2.5.2 (SDL 2.28.3, Python 3.11.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


# 2048 Simulation Project Documentation

## Features

### 1. Game Initialization
- The `Game` class is designed to simulate the 2048 game.
- It includes an optional graphical user interface (GUI) using the Pygame library.
- Users can specify whether to render the GUI for visual representation or run in a non-GUI mode for testing purposes.

### 2. Game Board
- The game board is represented as a grid with customizable dimensions (`GRID_SIZE`) = 4.
- 2 tiles are initialized with random values (2 or 4) upon starting the game.

### 3. User Interface (GUI)
- If the GUI is enabled, the simulation renders the game window using Pygame.
- Tiles are displayed with appropriate colors, and their values are shown in the center.
- A scoring system is implemented to keep track of the user's progress.

### 4. Tile Movement
- Users can move tiles in four directions: up, down, left, and right.
- Tiles slide as far as possible in the specified direction, merging with adjacent tiles of the same value.

### 5. Random Tile Generation
- New tiles are added to the grid randomly, with a 90% probability of a 2 and a 10% probability of a 4.
- This randomness introduces a dynamic element to the game.

### 6. Scoring
- The score is calculated based on the values of merged tiles during movements.
- The goal is to achieve the highest score possible through strategic tile merging.

### 7. Game Over Detection
- The simulation checks for game-over conditions after each move.
- Game over is determined if there are no empty tiles or if no adjacent tiles have the same value.
- In GUI mode, a "Game Over" message is displayed on the window when the game concludes.

## Usage

### Instantiation
```python
game = Game(gui=True)  # Creates a new game with GUI
# OR
game = Game(gui=False)  # Creates a new game without GUI
```

### Game Actions
- `game.move(direction)`: Performs a move in the specified direction (0: up, 1: right, 2: down, 3: left).
- `game.reset()`: Resets the game board to its initial state.
- `game.check_game_over()`: Checks if the game is over.

### Example
```python
game = Game()
game.move(1)  # Move tiles to the right
game.render()  # Render the updated game state
```

### Notes
- The `__str__` method provides a string representation of the current game state.

## Dependencies
- Pygame library (for GUI): Install via `pip install pygame`.


In [3]:
class Game:
    def __init__(self, gui=True, grid=None):
        # the gui flag is used to determine whether to render the game or not (might be useful for testing)
        # Initialize the game
        if gui:
            self.gui = True
            pygame.init()
            self.window = pygame.display.set_mode((WINDOW_SIZE, WINDOW_SIZE))
            self.font = pygame.font.SysFont("Arial", 32)
        if not gui:
            self.gui = False
        self.score = 0
        if grid:
            self.grid = grid
        else:
            self.reset()

    def reset(self):
        self.grid = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        self.add_random_tile()
        self.add_random_tile()
        self.score = 0

    def add_random_tile(self):
        # Add a random tile to the grid, probability of adding a 2 is 90% and 4 is 10%
        empty_tiles = []
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                if self.grid[i][j] == 0:
                    empty_tiles.append((i, j))
        if empty_tiles:
            i, j = random.choice(empty_tiles)
            self.grid[i][j] = 2 if random.random() < 0.9 else 4

    def render(self):
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                # draw the tile
                pygame.draw.rect(self.window, TILE_COLORS[self.grid[i][j]], (j * TILE_SIZE, i * TILE_SIZE, TILE_SIZE, TILE_SIZE))

                # add the text to the tile
                text = self.font.render(str(self.grid[i][j]) if self.grid[i][j] else "", True, (119, 110, 101))
                text_rect = text.get_rect()
                text_rect.center = (j * TILE_SIZE + TILE_SIZE / 2, i * TILE_SIZE + TILE_SIZE / 2)
                self.window.blit(text, text_rect)

                # add a border around the tiles
                pygame.draw.rect(self.window, (187, 173, 160), (j * TILE_SIZE, i * TILE_SIZE, TILE_SIZE, TILE_SIZE), 5)

        pygame.display.update()

    def move(self, direction):
        # direction: 0 - up, 1 - right, 2 - down, 3 - left
        # slide the tiles in the given direction
        merged = [[False for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        initial_grid = [row[:] for row in self.grid]

        # Slide tiles as far as possible in the given direction, merging tiles of the same value once.
        if direction == 0:
            for i in range(GRID_SIZE):
                for j in range(GRID_SIZE):
                    shift = 0
                    for k in range(i): # k checks all tiles between i and 0
                        if self.grid[k][j] == 0:
                            shift += 1
                    
                    if shift:
                        self.grid[i - shift][j] = self.grid[i][j]
                        self.grid[i][j] = 0

                    if i - shift - 1 >= 0 and self.grid[i - shift - 1][j] == self.grid[i - shift][j] and not merged[i - shift - 1][j] and not merged[i - shift][j]:
                        self.score += self.grid[i - shift][j] * 2
                        self.grid[i - shift - 1][j] *= 2
                        self.grid[i - shift][j] = 0
                        merged[i - shift - 1][j] = True

        elif direction == 1:
            for i in range(GRID_SIZE):
                for j in range(GRID_SIZE - 1, -1, -1):
                    shift = 0
                    for k in range(j + 1, GRID_SIZE):
                        if self.grid[i][k] == 0:
                            shift += 1

                    if shift:
                        self.grid[i][j + shift] = self.grid[i][j]
                        self.grid[i][j] = 0

                    if j + shift + 1 < GRID_SIZE and self.grid[i][j + shift + 1] == self.grid[i][j + shift] and not merged[i][j + shift + 1] and not merged[i][j + shift]:
                        self.score += self.grid[i][j + shift] * 2
                        self.grid[i][j + shift + 1] *= 2
                        self.grid[i][j + shift] = 0
                        merged[i][j + shift + 1] = True

        elif direction == 2:
            for i in range(GRID_SIZE - 1, -1, -1):
                for j in range(GRID_SIZE):
                    shift = 0
                    for k in range(i + 1, GRID_SIZE):
                        if self.grid[k][j] == 0:
                            shift += 1

                    if shift:
                        self.grid[i + shift][j] = self.grid[i][j]
                        self.grid[i][j] = 0

                    if i + shift + 1 < GRID_SIZE and self.grid[i + shift + 1][j] == self.grid[i + shift][j] and not merged[i + shift + 1][j] and not merged[i + shift][j]:
                        self.score += self.grid[i + shift][j] * 2
                        self.grid[i + shift + 1][j] *= 2
                        self.grid[i + shift][j] = 0
                        merged[i + shift + 1][j] = True

        elif direction == 3:
            for i in range(GRID_SIZE):
                for j in range(GRID_SIZE):
                    shift = 0
                    for k in range(j):
                        if self.grid[i][k] == 0:
                            shift += 1

                    if shift:
                        self.grid[i][j - shift] = self.grid[i][j]
                        self.grid[i][j] = 0

                    if j - shift - 1 >= 0 and self.grid[i][j - shift - 1] == self.grid[i][j - shift] and not merged[i][j - shift - 1] and not merged[i][j - shift]:
                        self.score += self.grid[i][j - shift] * 2
                        self.grid[i][j - shift - 1] *= 2
                        self.grid[i][j - shift] = 0
                        merged[i][j - shift - 1] = True

        if self.grid != initial_grid:
            self.add_random_tile()
            if self.check_game_over():
                if self.gui:
                    pygame.time.delay(5000)
                    pygame.quit()
                return (1, self.grid)
            if self.gui:
                self.render()
            return (0, self.grid)
        else:
            return (2, self.grid)

    def check_game_over(self):
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                if self.grid[i][j] == 0:
                    return
                if i > 0 and self.grid[i - 1][j] == self.grid[i][j]:
                    return
                if i < GRID_SIZE - 1 and self.grid[i + 1][j] == self.grid[i][j]:
                    return
                if j > 0 and self.grid[i][j - 1] == self.grid[i][j]:
                    return
                if j < GRID_SIZE - 1 and self.grid[i][j + 1] == self.grid[i][j]:
                    return

        # overlay text on the screen
        if self.gui:
            text = self.font.render("Game Over!", True, (119, 110, 101))
            text_rect = text.get_rect()
            text_rect.center = (WINDOW_SIZE / 2, WINDOW_SIZE / 2)
            self.window.blit(text, text_rect)
            pygame.display.update()
        return True
        

    def __str__(self):
        return "\n".join([" ".join([str(self.grid[i][j]) for j in range(GRID_SIZE)]) for i in range(GRID_SIZE)])

### Run the cell below to play the game manually

Game object uses pygame to display and in the background it uses the class to work

It has a move function which takes the direction as input and moves the tiles in that direction
Possible directions are: 'up', 'down', 'left', 'right' which are represented by the 0, 2, 3, 1 respectively




In [4]:
game = Game()
game.render()

# close the game window when the user presses the close button
done = False
while not done:
    # game loop
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.display.quit()
            pygame.quit()
            print("Exiting...")
            done = True
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                game.reset()
                game.render()
            elif event.key == pygame.K_UP:
                if game.move(0)[0] == 1:
                    done = True
            elif event.key == pygame.K_RIGHT:
                if game.move(1)[0] == 1:
                    done = True
            elif event.key == pygame.K_DOWN:
                if game.move(2)[0] == 1:
                    done = True
            elif event.key == pygame.K_LEFT:
                if game.move(3)[0] == 1:
                    done = True
        else:
            continue

Exiting...


### Random moves simulation

Done to demonstrate the game is working properly


In [5]:
game = Game()
game.render()

done = False
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.display.quit()
            pygame.quit()
            print("Exiting...")
            done = True
    pygame.time.delay(200)
    if game.move(random.randint(0, 3))[0] == 1:
        done = True
        

### Monte Carlo Tree Search (MCTS)
**Monte Carlo Tree Search (MCTS) Overview**

Monte Carlo Tree Search (MCTS) is a popular algorithm used in decision-making processes, particularly in artificial intelligence for games and optimization problems. It is commonly employed when the full state space is too vast to explore exhaustively.

### Key Concepts:

1. **Tree Search:**
   - MCTS builds a tree structure representing possible sequences of moves in a game or actions in a decision-making scenario.

2. **Monte Carlo Simulation:**
   - The algorithm relies on random sampling (Monte Carlo simulations) to explore parts of the decision space, gradually refining its understanding of the most promising choices.

3. **Four Key Steps:**
   - **Selection:** Starting from the root of the tree, traverse down the tree based on certain criteria (often using UCT - Upper Confidence Bounds for Trees) to find a promising node.
   - **Expansion:** Expand the selected node by adding one or more child nodes representing possible moves or actions.
   - **Simulation:** Conduct a Monte Carlo simulation from the newly added node. This involves making random moves or decisions until a terminal state is reached.
   - **Backpropagation:** Update the statistics (e.g., visit count, total reward) of the nodes visited during the simulation. Propagate this information up the tree.

### MCTS in the Context of 2048 Solver:

- **Selection:** The algorithm selects moves (up, down, left, right) based on accumulated statistics, favoring moves that lead to higher scores in the simulations.
  
- **Expansion:** For each selected move, the algorithm explores possible outcomes by adding child nodes representing different game states.

- **Simulation:** The `random_policy` function simulates multiple random games from each newly added node to estimate the potential outcomes of the move.

- **Backpropagation:** The results of the simulations are used to update the statistics of the nodes visited during the selection and expansion steps.

### Purpose in 2048 Solver:

- **Optimal Move Selection:** MCTS helps the AI make informed decisions on the best move in a given game state.
  
- **Dynamic Decision Making:** By using random simulations, the algorithm adapts to the dynamic nature of the game, providing a balance between exploration and exploitation.

- **Efficient Exploration:** Instead of exhaustively searching the entire decision space, MCTS focuses on promising regions, making it suitable for scenarios with large and uncertain state spaces.

In the provided 2048 solver, MCTS is applied iteratively to determine the best move that maximizes the overall score based on the outcomes of simulated games.

In [6]:
def random_policy(game):
    game_copy = copy.deepcopy(game)
    while not game_copy.check_game_over():
        game_copy.move(random.randint(0, 3))
    return game_copy.score, max(max(row) for row in game_copy.grid)

def mcts(initial_game):
    urdl_score = [0, 0, 0, 0]

    for move in range(4):
        game_copy = copy.deepcopy(initial_game)
        result, grid = game_copy.move(move)

        if result == 2:
            continue

        # try random policy for 100 games
        for i in range(80):
            output = random_policy(game_copy)

            urdl_score[move] += output[0]
            # urdl_score_max_tile[move] += output[1]

    best_move_by_score = urdl_score.index(max(urdl_score))
    # best_move_by_max_tile = urdl_score_max_tile.index(max(urdl_score_max_tile))

    initial_game.move(best_move_by_score)
    # print("Game State:\n", str(initial_game))

def monte_carlo_simulation(initial_game):
    game = copy.deepcopy(initial_game)
    run = 1
    while not game.check_game_over():
        mcts(game)
        run += 1

    print("Final State:\n", str(game))
    print("Score: " + str(game.score))
    print("Max Tile: " + str(max(max(row) for row in game.grid)))

    return game.score, max(max(row) for row in game.grid), game.grid
    


# AI Solver using Monte Carlo Tree Search (MCTS) for 2048

## Overview

The AI agent for 2048 is implemented using the Monte Carlo Tree Search algorithm. This solver aims to make informed decisions about the best move in a given game state by simulating multiple random games and choosing the move that leads to the highest overall score.

## Components

### 1. Random Policy

- The `random_policy` function simulates random moves in a given game until a game-over state is reached.
- The score and the maximum tile value achieved in the simulated games are recorded.

### 2. MCTS (Monte Carlo Tree Search)

- The `mcts` function performs Monte Carlo Tree Search for each possible move (up, down, left, right).
- It utilizes the `random_policy` to simulate multiple games and accumulate scores for each move.
- The move with the highest accumulated score is selected as the best move.

### 3. Monte Carlo Simulation

- The `monte_carlo_simulation` function applies the MCTS algorithm iteratively until a game-over state is reached.
- It prints the final state of the game, the total score, and the maximum tile value achieved.

## Usage

```python
# Create an instance of the Game class
initial_game = Game(gui=False)  # Set gui=True for visualization

# Run Monte Carlo Simulation
score, max_tile, final_grid = monte_carlo_simulation(initial_game)

# Output results
print("Score: " + str(score))
print("Max Tile: " + str(max_tile))
print("Final State:\n", str(final_grid))
```

## Notes

- Adjust the number of iterations in the `random_policy` function based on computational resources and desired accuracy.
- The code assumes the existence of the `Game` class with the `move`, `check_game_over`, and other relevant methods.

## Future Improvements

- Fine-tune parameters such as the number of iterations in the random policy and explore other enhancements to the MCTS algorithm.
- Implement optimizations for efficiency, especially when dealing with a large number of simulations.

Feel free to integrate this documentation into your project and customize it as needed.

In [7]:

Sum = 0
Max = 0
Max_game = None
_2prob = 0
_4prob = 0
_8prob = 0
_16prob = 0
_32prob = 0
_64prob = 0
_128prob = 0
_256prob = 0
_512prob = 0
_1024prob = 0
_2048prob = 0
_4096prob = 0

init_time = time.time()
for i in range(10):
    game = Game(gui=False)
    (score, max_tile, grid) = monte_carlo_simulation(game)
    if any(4096 in row for row in grid):
        _4096prob += 1
    if any(2048 in row for row in grid):
        _2048prob += 1
    if any(1024 in row for row in grid):
        _1024prob += 1
    if any(512 in row for row in grid):
        _512prob += 1
    if any(256 in row for row in grid):
        _256prob += 1
    if any(128 in row for row in grid):
        _128prob += 1
    if any(64 in row for row in grid):
        _64prob += 1
    if any(32 in row for row in grid):
        _32prob += 1
    if any(16 in row for row in grid):
        _16prob += 1
    if any(8 in row for row in grid):
        _8prob += 1
    if any(4 in row for row in grid):
        _4prob += 1
    if any(2 in row for row in grid):
        _2prob += 1

print("\nTime taken (s): " + str(time.time() - init_time))

print("\nAverage Score: " + str(Sum / 100))
print("Max Score: " + str(Max))
print("2: " + str(_2prob/100))
print("4: " + str(_4prob/100))
print("8: " + str(_8prob/100))
print("16: " + str(_16prob/100))
print("32: " + str(_32prob/100))
print("64: " + str(_64prob/100))
print("128: " + str(_128prob/100))
print("256: " + str(_256prob/100))
print("512: " + str(_512prob/100))
print("1024: " + str(_1024prob/100))
print("2048: " + str(_2048prob/100))
print("4096: " + str(_4096prob/100))


Final State:
 4 8 2 4
2 512 64 8
4 256 4096 4
8 64 8 2
Score: 50724
Max Tile: 4096
